In [1]:
from functions import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import chi2_contingency
from scipy.stats.contingency import association

In [2]:
url_1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt'
url_2 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt'
url_3 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt'
url_4 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt'

In [3]:
client_profile = pd.read_csv(url_1)
exp_client = pd.read_csv(url_2)

In [4]:
df_1 = pd.read_csv(url_3, parse_dates=['date_time'])
df_2 = pd.read_csv(url_4, parse_dates=['date_time'])

In [5]:
# Concat Digital Footprints (df_final_web_data) pt_1 and pt_2 into one file:
df = pd.concat([df_1, df_2])

# Remove duplicated rows in merged digital footprints file 'df'
# df.duplicated().sum() - there are 10764 duplicated rows
df.drop_duplicates(inplace=True)

In [6]:
# Merge client profiles file (df_final_demo.txt) with experiment roster (df_final_experiment_clients.txt)

clients = pd.merge(client_profile, exp_client, on='client_id', how='outer')

In [7]:
# Cleaning 'clients' file:
# 1. Fill rows wiht NaN value in 'Variation' column with 'Uknown' - 20109 rows
# 2. Drop row with NaN value in 'gender' column - 12 rows
# 3. Drop rows with 'X' value in 'gender column' - 2 rows
# 4. Drop row with NaN value in 'age' column - 1 row

#1
#clients = clients.dropna(subset=['Variation']) 
clients['Variation'] = clients['Variation'].fillna('Unknown')

#2
clients = clients.dropna(subset=['gendr']) 

#3
clients.drop(clients[clients['gendr'] == 'X'].index, inplace=True)

#4
clients = clients.dropna(subset=['clnt_age']) 

In [8]:
df_time , df_avg_step_durations = calc_avg_step_duration(df)

df_time_errors, df_client_change = detect_process_changes(df_time)

df_client_visits = nr_of_visits(df)

df_client_confirms = nr_of_each_step(df)

clients_who_confirmed = clients_who_finished(df)

clients_who_confirmed = clients_who_confirmed.set_index('client_id').sort_values(by='client_id')

In [9]:
# Call funtion 'data_summary' to create data frame with web data summary

dfs =[df_avg_step_durations, df_client_change, df_client_visits, clients_who_confirmed, df_client_confirms]

web_data_summary = data_summary(dfs)

web_data_summary['avg_step_duration'] = pd.to_timedelta(web_data_summary['avg_step_duration'])

In [12]:
# Merge web_data_summary with 'clients' data frame

whole = pd.merge(clients, web_data_summary, on='client_id', how='outer')

# Drop rows with NaN values
whole = whole.dropna(subset='Variation')

In [15]:
# Use only data for 'Test' and 'Control' groups

filtered = whole[whole['Variation'].isin(['Test', 'Control'])]


In [18]:
#CSV Generation: 
filtered_csv_link = 'data/src/web_data_summary.csv'
filtered.to_csv(filtered_csv_link, index=False)

In [69]:
# Combine df_time_errors (web data footprint + additional columns with time and error counts) with clients table
# Remove rows with NaN values in 'Variation' column (so we don't know if these are clients from test or control groups)

total_web_df = pd.merge(df_time_errors, clients, on='client_id', how='left')
total_web_df = total_web_df.dropna(subset='Variation')


In [71]:
#CSV Generation of total_web_df: 
total_web_df_csv_link = 'data/src/total_web_data_df.csv'
total_web_df.to_csv(total_web_df_csv_link, index=False)